In [1]:
# Packages to be used
import networkx as nx
from networkx.algorithms import approximation
from networkx.algorithms import community
from networkx.algorithms import centrality
import numpy as np
import matplotlib.pyplot as plt
import ndlib.models.epidemics as ep
import ndlib.models.ModelConfig as mc
import random
# import scipy
# import math
# import sys
import pandas as pd
# from fractions import Fraction
# import timeit
# from time import sleep
import igraph as ig
import time
import collections
# import heapq as heap
# from tqdm import tqdm 
import community
from ndlib.viz.mpl.DiffusionTrend import DiffusionTrend

no display found. Using non-interactive Agg backend


In [2]:
# Load datasets, outside of definitions

# G1 = nx.read_edgelist("Wiki-Vote.txt", create_using=nx.DiGraph(), nodetype= int)
# print(G1)
G2 = nx.read_edgelist("lastfm_asia_edges.txt", create_using=nx.Graph(), nodetype=int)
print(G2)


Graph with 7624 nodes and 27806 edges


In [ ]:
# Making a directed graph undirected

G1_undirected = G1.to_undirected()
print(G1_undirected)

Heuristics to block/remove nodes

In [3]:
# Centrality measures

# Degree
# sort_degree = sorted(dict(G2.degree).items(), key=lambda item: item[1], reverse=True)
# print(sort_degree)

# Degree centrality
# degree_centrality = nx.centrality.degree_centrality(G2)
# sort_degree_centrality = sorted(degree_centrality.items(), key=lambda item: item[1], reverse=True)
print("Done with degree centrality")

# Betweenness

betweenness = nx.betweenness_centrality(G2)
sort_betweenness = sorted(betweenness.items(), key=lambda item: item[1], reverse=True)
# print("Done with betweenness centrality")


# # Closeness

# closeness = nx.closeness_centrality(G2)
# sort_closeness = sorted(closeness.items(), key=lambda item: item[1], reverse=True)
# print("Done with closeness centrality")


# # Influence maximization
# # Pagerank

# pagerank = nx.pagerank(G2)
# sort_pagerank = sorted(pagerank.items(), key=lambda item: item[1], reverse=True)
# print("Done with pagerank centrality")

# # Eigenvector

# eigenvector = nx.eigenvector_centrality(G2)
# sort_eigenvector = sorted(eigenvector.items(), key=lambda item: item[1], reverse=True)
# print("Done with eigenvector centrality")

Done with degree centrality


In [4]:
# To only get the keys from the sorted list

only_keys = [k for k,v in sort_betweenness]
print(only_keys)

# Take the top 50 nodes to be used as seeds later
first_100 = only_keys[0:100]
print(first_100)

[7199, 7237, 2854, 4356, 6101, 5454, 4338, 5127, 3450, 4785, 3530, 3103, 3544, 2510, 2600, 4811, 4900, 1795, 1376, 5274, 3584, 4146, 524, 3873, 3181, 5370, 7339, 2707, 5578, 1689, 1792, 1897, 1698, 6519, 3855, 3240, 7100, 5646, 645, 1250, 1937, 1618, 7352, 543, 3130, 3597, 1799, 3210, 766, 6478, 5854, 290, 2892, 2229, 1348, 891, 4125, 1677, 1849, 3039, 3653, 5110, 110, 3571, 167, 1312, 4319, 6241, 6891, 2337, 3400, 985, 3381, 7001, 7162, 5441, 4359, 5653, 3751, 5876, 2473, 4941, 6617, 1812, 3038, 6463, 1464, 4039, 3330, 3701, 2422, 4880, 4206, 5725, 1875, 2470, 998, 4000, 1271, 6712, 221, 1706, 3073, 3341, 2798, 1300, 5095, 3325, 1005, 1334, 1462, 3080, 2961, 898, 4121, 4876, 5335, 2289, 3845, 4691, 6419, 3822, 244, 4384, 5461, 7351, 6446, 6564, 897, 6252, 2020, 3853, 2140, 2713, 212, 1316, 6108, 1850, 3360, 1384, 3921, 7028, 7565, 7595, 3670, 3504, 113, 4563, 5047, 6365, 7170, 5516, 2624, 4033, 7141, 7226, 7032, 4257, 5060, 4994, 2265, 4447, 3979, 2988, 1509, 380, 1230, 5534, 5065, 15

In [ ]:
# Add weight to the edges of the graph. To be used as thresholds for the ICM

degree_dict = dict(G2.degree())
weights = []
for u, v, data in G2.edges(data=True):
    ratio = 1 / degree_dict[v]
    data['weight'] = ratio
    weights.append(ratio)

print(weights)

In [5]:
# Create a copy of the original graph to be modified later, by removing nodes
G2_copy = G2.copy()


G2_copy.remove_nodes_from(first_100)
print(G2_copy)

Graph with 7524 nodes and 21874 edges


The following is the centrality measures for the new graph (After removing the nodes)

In [6]:
# Centrality measures

# Degree
# sort_degree = sorted(dict(G2.degree).items(), key=lambda item: item[1], reverse=True)
# print(sort_degree)

# Degree centrality
degree_centrality_after = nx.centrality.degree_centrality(G2_copy)
sort_degree_centrality_after = sorted(degree_centrality_after.items(), key=lambda item: item[1], reverse=True)
# print(sort_degree_centrality)

# Betweenness

betweenness_after = nx.betweenness_centrality(G2_copy)
sort_betweenness_after = sorted(betweenness_after.items(), key=lambda item: item[1], reverse=True)
# print(sort_betweenness)

# Closeness

closeness_after = nx.closeness_centrality(G2_copy)
sort_closeness_after = sorted(closeness_after.items(), key=lambda item: item[1], reverse=True)
# # print(sort_closeness)


# # Influence maximization
# # Pagerank

pagerank_after = nx.pagerank(G2_copy)
sort_pagerank_after = sorted(pagerank_after.items(), key=lambda item: item[1], reverse=True)
# # print(sort_pagerank)

# # Eigenvector

eigenvector_after = nx.eigenvector_centrality(G2_copy)
sort_eigenvector_after = sorted(eigenvector_after.items(), key=lambda item: item[1], reverse=True)
# # print(sort_eigenvector)

In [11]:
# To only get the keys from the sorted list

only_keys_after = [k for k,v in sort_betweenness_after]
print(only_keys_after)

# Take the top 50 nodes to be used as seeds later
first_100_after = only_keys_after[0:100]
print(first_100_after)

[5095, 1300, 3504, 1509, 1462, 7030, 1005, 6446, 3251, 1384, 5065, 3360, 1230, 7488, 3853, 2020, 1706, 1011, 5047, 1956, 2488, 7226, 2289, 2091, 2798, 2694, 7565, 3582, 299, 3341, 1691, 3317, 5314, 2299, 4916, 2122, 3822, 3994, 1945, 5543, 763, 2140, 7595, 1316, 381, 3073, 4929, 2961, 1281, 113, 4309, 7224, 5944, 5060, 5463, 2897, 5564, 54, 6022, 4556, 933, 7165, 3279, 6252, 1381, 4426, 2056, 5510, 1724, 6408, 3833, 2304, 216, 2265, 2024, 4994, 6564, 4335, 2805, 221, 7028, 2170, 7084, 5072, 3948, 2713, 2988, 3418, 7170, 3898, 2386, 5706, 4908, 2591, 2889, 212, 6339, 213, 5857, 1506, 5534, 5703, 4119, 6217, 3502, 6621, 4934, 378, 6323, 3081, 5610, 6514, 5526, 2624, 4623, 1665, 6219, 3845, 3977, 421, 1986, 958, 3969, 2994, 6460, 2142, 4470, 1752, 2615, 1350, 6194, 166, 7284, 566, 1627, 4531, 989, 7601, 708, 1941, 5105, 5063, 1, 2399, 374, 6415, 3670, 3158, 4033, 2995, 6441, 4563, 4931, 1817, 1963, 647, 1334, 4673, 3829, 4549, 7213, 667, 3755, 1057, 2816, 7141, 5793, 3585, 3350, 3991, 18,

In [ ]:
# Adding weight to the edges of the graph

# for u,v in G1.edges():
#     G1[u][v]['weight'] = 1/G1.in_degree(v)
# G1.get_edge_data(30,1412)
# G1.in_degree(1412)

# for u,v in G2.edges():
#     G2[u][v]['weight'] = 1/G2.degree(v)
# G2.get_edge_data(0,5)

degree_dict = dict(G2_copy.degree())
weights = []
for u, v, data in G2_copy.edges(data=True):
    ratio = 1 / degree_dict[v]
    data['weight'] = ratio
    weights.append(ratio)

In [ ]:
# Adds an attribute to the nodes of the graph, for example degree centrality

nx.set_node_attributes(G1, centrality, "centrality")
# nx.set_node_attributes(G2, centrality, "centrality")

In [ ]:
# Filtering the graph by removing edges based on their weight attribute

for u,v in list(G1.edges()):
    if G1[u][v]['weight'] < 0.04:
        G1.remove_edge(u,v)
G1_new = G1.copy()

# Remove nodes based on their degree

for u in list(G1.nodes()):
    if G1.degree(u)< 10:
        G1.remove_node(u)

# Remove nodes based on their degree centrality

for u,v in centrality.items():
    if v < 0.004:
        G1.remove_node(u)

In [ ]:
# Lists of sources and targets based on in/out degrees of nodes

sources = []
targets = []
for u in G1.nodes():
    if G1.in_degree(u) == 0:
        sources.append(u)
for v in G1.nodes():
    if G1.out_degree(v) == 0:
        targets.append(v)

In [ ]:
# Splitting the sources to smaller components (useful for seed selection)

sub_sources = sources[0:50]
print(sub_sources)


In [ ]:
import random
def random(nodes, n):
    ''' Select seeds randomly
    Args:
        nodes (list) [#node]: node list of the graph;
        n (int): number of seeds;
    Returns: 
        seeds: (list) [#seed]: selected seed nodes index;
    '''
    # random.sample(nodes, n)
    # return nodes[:n]
    # np.random.shuffle(nodes)
    # return nodes[:n]
    seeds = random.sample(nodes, n)
    return seeds

In [ ]:
def mia(nodes, edges, n):
    ''' select seeds by mia policy
    args:
        nodes (list) [#node]: node list of the graph;
        edges (list of list) [#edge, 2]: edge list of the graph;
        n (int): number of seeds;
    returns: 
        seeds: (list) [#seed]: selected seed nodes index;
    '''
    out_connection = {}
    in_connection = {}
    centrality_score = {}
    seeds = []
    for edge in edges:
        if edge[0] in out_connection:
            out_connection[edge[0]].append(edge[1])
        else:
            out_connection[edge[0]] = [edge[1]]
        if edge[1] in in_connection:
            in_connection[edge[1]] += 1
        else:
            in_connection[edge[1]] = 1
    for node in nodes:
        centrality_score[node] = mia_centrality(node, out_connection, in_connection)
    i = 0
    for node, _ in sorted(centrality_score.items(), key=lambda item: item[1], reverse=True):
        if i >= n:
            break
        else:
            i += 1
        seeds.append(node)
    return seeds

def mia_centrality(node, out_connection, in_connection):
    ''' select seeds by mia centrality policy
    '''
    theta = 0.5
    c_score = 0
    q = 1  # threshold of influence
    visited = set()
    path_prob = 1
    c_score = dfs(visited, out_connection, path_prob, in_connection, node, theta, q)
    return c_score

def dfs(visited, out_connection, path_prob, in_connection, node, theta, q):
    if node not in visited:
        visited.add(node)
        if node in out_connection:
            for neighbour in out_connection[node]:
                path_prob *= q / in_connection[neighbour]
                if path_prob >= theta:
                    dfs(visited, out_connection, path_prob, in_connection, neighbour, theta, q)
                    path_prob /= (q / in_connection[neighbour])
                else:
                    path_prob /= (q / in_connection[neighbour])
    N_of_nodes = len(visited)
    return N_of_nodes

In [ ]:
def degree(edges, n):
    ''' Select seeds by degree policy
    Args:
        edges (list of list) [#edge, 2]: edge list of the graph;
        n (int): number of seeds;
    Returns: 
        seeds: (list) [#seed]: selected seed nodes index;
    '''
    degree = {}
    seeds = []
    for edge in edges:
        if edge[0] in degree:
            degree[edge[0]] += 1
        else:
            degree[edge[0]] = 1
        if edge[1] in degree:
            degree[edge[1]] += 1
        else:
            degree[edge[1]] = 1
    seeds = list({k: v for k, v in sorted(degree.items(), key=lambda item: item[1], reverse=True)}.keys())[:n]
    return seeds

In [ ]:
def degree_discount(edges, n):
    ''' Select seeds by degree discount degree
    Args:
        edges (list of list) [#edge, 2]: edge list of the graph;
        n (int): number of seeds;
    Returns: 
        seeds: (list) [#seed]: selected seed nodes index;
    '''
    out_degree = {}
    connection = {}
    seeds = []
    for edge in edges:
        if edge[1] in connection:
            connection[edge[1]].append(edge[0])
        else:
            connection[edge[1]] = [edge[0]]
        if edge[0] in out_degree:
            out_degree[edge[0]] += 1
        else:
            out_degree[edge[0]] = 1
    while len(seeds) < n:
        seed = sorted(out_degree.items(), key=lambda item: item[1], reverse=True)[0][0]
        seeds.append(seed)
        out_degree[seed] = -1
        if seed in connection:
            for node in connection[seed]:
                if node in out_degree:
                    out_degree[node] -= 1
    return seeds

In [ ]:
def degree_neighbor_fix(edges, n):
    ''' select seeds by degree neighbor fix policy
    args:
        edges (list of list) [#edge, 2]: edge list of the graph;
        n (int): number of seeds;
    returns: 
        seeds: (list) [#seed]: selected seed nodes index;
    '''
    out_degree = {}
    centrality_score = {}
    connection = {}
    seeds = []
    for edge in edges:
        if edge[1] in connection:
            connection[edge[1]].append(edge[0])
        else:
            connection[edge[1]] = [edge[0]]
        if edge[0] in out_degree:
            out_degree[edge[0]] += 1
        else:
            out_degree[edge[0]] = 1
    centrality_score = out_degree.copy()
    for edge in edges:
        if edge[1] in out_degree:
            centrality_score[edge[0]] += out_degree[edge[1]]
    while len(seeds) < n:
        seed = sorted(centrality_score.items(), key=lambda item: item[1], reverse=True)[0][0]
        seeds.append(seed)
        centrality_score[seed] = -1
        if seed in connection:
            for node in connection[seed]:
                if node in out_degree:
                    centrality_score[node] -= out_degree[seed]

    return seeds

In [ ]:
def degree_neighbor(edges, n):
    ''' select seeds by degree neighbor policy
    args:
        edges (list of list) [#edge, 2]: edge list of the graph;
        n (int): number of seeds;
    returns: 
        seeds: (list) [#seed]: selected seed nodes index;
    '''
    out_degree = {}
    centrality_score = {}
    seeds = []
    for edge in edges:
        if edge[0] in out_degree:
            out_degree[edge[0]] += 1
        else:
            out_degree[edge[0]] = 1
    centrality_score = out_degree.copy()
    for edge in edges:
        if edge[1] in out_degree:
            centrality_score[edge[0]] += out_degree[edge[1]]
    seeds = sorted(centrality_score.items(), key=lambda item: item[1], reverse=True)
    return seeds


In [ ]:
# Create communities based on the louvain method

louvain = nx.algorithms.community.louvain_communities(G2)
print(len(louvain))
# print(louvain[131])

In [ ]:
# Create a seed set based on the communities and a heuristic

seeds_new = {}
for community in louvain:
    max_node = max(community, key=lambda node: degree_centrality[node])
    seeds_new[max_node] = community

In [ ]:
# policy = 'degree'

# seeds_number = int(len(G1.nodes) * init_rate)
# seeds = list({k: v for k, v in sorted(dict(degreenew).items(), key=lambda item: item[1], reverse=True)}.keys())[:seeds_number]
policy = 'mia'

init_rate = 0.02
# nodes = G2_new.nodes
# edges = G2_new.edges
# # nodes = G_new.vs
# # edges = G_new.es
nodes = G2.nodes()
edges = G2.edges()
seeds_number = int(len(nodes) * init_rate)

if policy == 'degree':
    seeds = degree(edges, seeds_number)
    # seeds = list({k: v for k, v in sorted(dict(new_degree).items(), key=lambda item: item[1], reverse=True)}.keys())[:seeds_number]
if policy == 'random':
    seeds = random(nodes, seeds_number)
if policy == 'degree_discount':
    seeds = degree_discount(edges, seeds_number)
if policy == 'mia':
   seeds = mia(nodes, edges, seeds_number) 
if policy == 'degree_neighbor_fix':
    seeds = degree_neighbor_fix(edges, seeds_number)
if policy == 'degree_neighbor':
    seeds = degree_neighbor(edges, seeds_number)
print(f'Number of seeds: {len(seeds)}')

In [ ]:
print(seeds)

Use louvain communities without fairness measure and different centrality measures

In [ ]:
import community

# Partition into communities
partition = community.best_partition(G2)
for node in G2.nodes:
    G2.nodes[node]['community'] = partition[node]
print(len(partition))

num_distinct_values = len(set(partition.values()))

print("The dictionary has", num_distinct_values, "distinct values.")

In [ ]:
# Calculate centralities (can also be done from the cell above, have to check to decide), for fairness this cell can be used to removed the selected nodes

seeds = set()
for community_id in set(partition.values()):
    community_nodes = [node for node in G2.nodes if G2.nodes[node]['community'] == community_id]
    max_degree_node = max(community_nodes, key=lambda node: pagerank[node])
    seeds.add(max_degree_node)
print("Number of seeds:", len(seeds))

In [ ]:
# Create a copy of the original graph to be modified later, by removing nodes
G2_copy_comm = G2.copy()


G2_copy_comm.remove_nodes_from(seeds)
print(G2_copy_comm)

In [ ]:
# Centrality measures

# Degree
# sort_degree = sorted(dict(G2.degree).items(), key=lambda item: item[1], reverse=True)
# print(sort_degree)

# Degree centrality
degree_centrality_after = nx.centrality.degree_centrality(G2_copy_comm)
sort_degree_centrality_after = sorted(degree_centrality_after.items(), key=lambda item: item[1], reverse=True)
# print(sort_degree_centrality)

# Betweenness

betweenness_after = nx.betweenness_centrality(G2_copy_comm)
sort_betweenness_after = sorted(betweenness_after.items(), key=lambda item: item[1], reverse=True)
# print(sort_betweenness)

# Closeness

closeness_after = nx.closeness_centrality(G2_copy_comm)
sort_closeness_after = sorted(closeness_after.items(), key=lambda item: item[1], reverse=True)
# print(sort_closeness)


# Influence maximization
# Pagerank

pagerank_after = nx.pagerank(G2_copy_comm)
sort_pagerank_after = sorted(pagerank_after.items(), key=lambda item: item[1], reverse=True)
# print(sort_pagerank)

# Eigenvector

eigenvector_after = nx.eigenvector_centrality(G2_copy_comm)
sort_eigenvector_after = sorted(eigenvector_after.items(), key=lambda item: item[1], reverse=True)
# print(sort_eigenvector)

In [ ]:
import community

# Partition into communities
partition = community.best_partition(G2_copy_comm)
for node in G2_copy_comm.nodes:
    G2_copy_comm.nodes[node]['community'] = partition[node]
print(len(partition))

num_distinct_values = len(set(partition.values()))

print("The dictionary has", num_distinct_values, "distinct values.")

In [ ]:
# Calculate centralities (can also be done from the cell above, have to check to decide)

seeds_after = set()
for community_id in set(partition.values()):
    community_nodes = [node for node in G2_copy_comm.nodes if G2_copy_comm.nodes[node]['community'] == community_id]
    max_degree_node = max(community_nodes, key=lambda node: degree_centrality_after[node])
    seeds_after.add(max_degree_node)
print("Number of seeds:", len(seeds_after))

Use louvain communities with equality fairness and different centrality measures

In [ ]:
import community

# Partition into communities
partition = community.best_partition(G2)
print(len(partition))
betweenness_centralities = {}
closeness_centralities = {}
degree_centralities = {}
eigenvector_centralities = {}
pagerank_centralities = {}
for community_id in set(partition.values()):
    community_nodes = [n for n in partition.keys() if partition[n] == community_id]
    community_subgraph = G2.subgraph(community_nodes)
    # betweenness_centralities[community_id] = nx.betweenness_centrality(community_subgraph)
    # degree_centralities[community_id] = nx.degree_centrality(community_subgraph)
    # closeness_centralities[community_id] = nx.closeness_centrality(community_subgraph)
    # eigenvector_centralities[community_id] = nx.eigenvector_centrality(community_subgraph)
    pagerank_centralities[community_id] = nx.pagerank(community_subgraph)

# Calculate fairness metric
fairness_metrics = {}
for community_id in set(partition.values()):
    # fairness = max(betweenness_centralities[community_id].values()) - min(betweenness_centralities[community_id].values())
    # fairness = max(degree_centralities[community_id].values()) - min(degree_centralities[community_id].values())
    # fairness = max(closeness_centralities[community_id].values()) - min(closeness_centralities[community_id].values())       
    # fairness = max(eigenvector_centralities[community_id].values()) - min(eigenvector_centralities[community_id].values())       
    fairness = max(pagerank_centralities[community_id].values()) - min(pagerank_centralities[community_id].values())                                                          
                                                   
    fairness_metrics[community_id] = fairness
    
# Calculate seeds from each community
seeds = []
min_fairness_community = min(fairness_metrics, key=fairness_metrics.get)

for community_id in set(partition.values()):
    community_nodes = [n for n in partition.keys() if partition[n] == community_id]
    community_subgraph = G2.subgraph(community_nodes)

    if community_id == min_fairness_community:
        # node = max(betweenness_centralities[community_id], key=betweenness_centralities[community_id].get)
        # node = max(degree_centralities[community_id], key=degree_centralities[community_id].get)
        # node = max(closeness_centralities[community_id], key=closeness_centralities[community_id].get)
        # node = max(eigenvector_centralities[community_id], key=eigenvector_centralities[community_id].get)
        node = max(pagerank_centralities[community_id], key=pagerank_centralities[community_id].get)

    else:
        # nodes_sorted = sorted(betweenness_centralities[community_id].items(), key=lambda x: x[1], reverse=True)
        # nodes_sorted = sorted(degree_centralities[community_id].items(), key=lambda x: x[1], reverse=True)
        # nodes_sorted = sorted(closeness_centralities[community_id].items(), key=lambda x: x[1], reverse=True)
        # nodes_sorted = sorted(eigenvector_centralities[community_id].items(), key=lambda x: x[1], reverse=True)
        nodes_sorted = sorted(pagerank_centralities[community_id].items(), key=lambda x: x[1], reverse=True)
        node = nodes_sorted[0][0]

    seeds.append(node)
print("Number of seeds:", len(seeds))

The "correct" implementation for the equality fairness is bellow (as implemented in Farnandi's paper)

In [ ]:
import community
# Calculcate communities using louvain method
partition = community.best_partition(G2)
for node in G2.nodes:
    G2.nodes[node]['community'] = partition[node]
print(len(partition))

num_distinct_values = len(set(partition.values()))

print("The dictionary has", num_distinct_values, "distinct values.")

In [ ]:
# Create a dictionary to store the population ratio of each community:

community_sizes = {}
for c in set(partition.values()):
    nodes_in_community = [n for n in partition.keys() if partition[n] == c]
    community_sizes[c] = len(nodes_in_community)

# Calculate the centralities for each node in the graph

# closeness_centrality = nx.closeness_centrality(G2)
# betweenness_centrality = nx.betweenness_centrality(G2)
# degree_centrality = nx.degree_centrality(G2)
# eigen_centrality = nx.eigenvector_centrality(G2)
# pagerank_centrality = nx.pagerank(G2)

# Create a dictionary to store the centralities for each community:

# community_closeness = {}
community_betweenness = {}
# community_degree = {}
# community_eigen = {}
# community_pagerank = {}
for node in G2.nodes():
    community = partition[node]
    # if community not in community_closeness:
    #     community_closeness[community] = 0
    if community not in community_betweenness:
        community_betweenness[community] = 0
    # if community not in community_degree:
    #     community_degree[community] = 0
    # if community not in community_eigen:
    #     community_eigen[community] = 0
    # if community not in community_pagerank:
    #     community_pagerank[community] = 0

    # community_closeness[community] += closeness[node]
    community_betweenness[community] += betweenness[node]
    # community_degree[community] += degree_centrality[node]
    # community_eigen[community] += eigenvector[node]
    # community_pagerank[community] += pagerank[node]


# Calculate the total centrality for each community

# for community in community_closeness:
#     community_closeness[community] /= community_sizes[community]
for community in community_betweenness:
    community_betweenness[community] /= community_sizes[community]
# for community in community_degree:
#     community_degree[community] /= community_sizes[community]
# for community in community_eigen:
#     community_eigen[community] /= community_sizes[community]
# for community in community_pagerank:
#     community_pagerank[community] /= community_sizes[community]

# Calculate the number of seeds for each community proportional to the population ratio using the centralities:
seeds = {}
for node in G2.nodes():
    community = partition[node]
    if community not in seeds:
        seeds[community] = []
        # if len(seeds[community]) < community_sizes[community] * community_closeness[community]:
        #     seeds[community].append(node)
        if len(seeds[community]) < community_sizes[community] * community_betweenness[community]:
            seeds[community].append(node)
        # if len(seeds[community]) < community_sizes[community] * community_degree[community]:
        #     seeds[community].append(node)
        # if len(seeds[community]) < community_sizes[community] * community_eigen[community]:
        #     seeds[community].append(node)
        # if len(seeds[community]) < community_sizes[community] * community_pagerank[community]:
        #     seeds[community].append(node)

print("Number of seeds:", len(seeds))

In [ ]:
# Create a copy of the original graph to be modified later, by removing nodes (fair version)
G2_copy_comm_fair = G2.copy()


G2_copy_comm_fair.remove_nodes_from(seeds)
print(G2_copy_comm_fair)

In [ ]:
import community
# Calculcate communities using louvain method
partition = community.best_partition(G2_copy_comm_fair)
for node in G2_copy_comm_fair.nodes:
    G2_copy_comm_fair.nodes[node]['community'] = partition[node]
print(len(partition))

num_distinct_values = len(set(partition.values()))

print("The dictionary has", num_distinct_values, "distinct values.")

In [ ]:
community_sizes = {}
for c in set(partition.values()):
    nodes_in_community = [n for n in partition.keys() if partition[n] == c]
    community_sizes[c] = len(nodes_in_community)

# Calculate the centralities for each node in the graph

closeness_centrality = nx.closeness_centrality(G2_copy_comm_fair)
betweenness_centrality = nx.betweenness_centrality(G2_copy_comm_fair)

In [ ]:
# Create a dictionary to store the population ratio of each community:

community_sizes = {}
for c in set(partition.values()):
    nodes_in_community = [n for n in partition.keys() if partition[n] == c]
    community_sizes[c] = len(nodes_in_community)

# Calculate the centralities for each node in the graph

# closeness_centrality = nx.closeness_centrality(G2_copy_comm_fair)
# betweenness_centrality = nx.betweenness_centrality(G2_copy_comm_fair)
# degree_centrality = nx.degree_centrality(G2_copy_comm_fair)
# eigen_centrality = nx.eigenvector_centrality(G2_copy_comm_fair)
pagerank_centrality = nx.pagerank(G2_copy_comm_fair)

# Create a dictionary to store the centralities for each community:

community_closeness = {}
community_betweenness = {}
community_degree = {}
community_eigen = {}
community_pagerank = {}
for node in G2_copy_comm_fair.nodes():
    community = partition[node]
    # if community not in community_closeness:
    #     community_closeness[community] = 0
    # if community not in community_betweenness:
    #     community_betweenness[community] = 0
    # if community not in community_degree:
    #     community_degree[community] = 0
    # if community not in community_eigen:
    #     community_eigen[community] = 0
    if community not in community_pagerank:
        community_pagerank[community] = 0

    # community_closeness[community] += closeness_centrality[node]
    # community_betweenness[community] += betweenness_centrality[node]
    # community_degree[community] += degree_centrality[node]
    # community_eigen[community] += eigen_centrality[node]
    community_pagerank[community] += pagerank_centrality[node]


# Calculate the total centrality for each community

# for community in community_closeness:
#     community_closeness[community] /= community_sizes[community]
# for community in community_betweenness:
#     community_betweenness[community] /= community_sizes[community]
# for community in community_degree:
#     community_degree[community] /= community_sizes[community]
# for community in community_eigen:
#     community_eigen[community] /= community_sizes[community]
for community in community_pagerank:
    community_pagerank[community] /= community_sizes[community]

# Calculate the number of seeds for each community proportional to the population ratio using the centralities:
seeds = {}
for node in G2_copy_comm_fair.nodes():
    community = partition[node]
    if community not in seeds:
        seeds[community] = []
        # if len(seeds[community]) < community_sizes[community] * community_closeness[community]:
        #     seeds[community].append(node)
        # if len(seeds[community]) < community_sizes[community] * community_betweenness[community]:
        #     seeds[community].append(node)
        # if len(seeds[community]) < community_sizes[community] * community_degree[community]:
        #     seeds[community].append(node)
        # if len(seeds[community]) < community_sizes[community] * community_eigen[community]:
        #     seeds[community].append(node)
        if len(seeds[community]) < community_sizes[community] * community_pagerank[community]:
            seeds[community].append(node)

print("Number of seeds:", len(seeds))

In [ ]:
seeds_random = [766, 2565, 457, 11, 1549, 2688, 1166, 1151, 1374, 311, 2485, 813, 1133, 3453, 1608, 24, 2967, 988, 1098, 2972, 789, 4967, 2237, 3447, 5524, 3352, 173, 3456, 68, 5079, 993, 2651, 36, 3641, 306, 3449, 3642, 1305, 2328, 784, 2871, 722, 2658, 3032, 5802, 3787, 122, 1210, 1542, 4045, 2326, 4310, 996, 5179, 5531, 1496, 5189, 600, 310, 825, 1319, 3026, 5697, 879, 1972, 2713, 6, 3614, 87, 5796, 826, 3394, 3028, 2135, 2686, 319, 5188, 1470, 2297, 4099, 312, 109, 663, 1055, 2256, 4632, 20, 2653, 72, 634, 3454, 5800, 1492, 1062, 1922, 2145, 1395, 737, 2660, 1167, 2674, 2842, 2790, 5543, 769, 1428, 2693, 3452, 1808, 4948, 3666, 2700, 3457, 432, 2560, 5002, 3615, 317, 2323, 2369, 3010, 922, 2229, 3976, 4483, 2371, 1521, 546, 3586, 4441, 1137, 172, 3568, 6305, 1583, 2940, 695, 308, 4065, 2474, 1522, 707]

print(len(seeds_random))

For the following code dont forget to change the graphs for before/after blocking the nodes

In [12]:
# LTM
# This code can be used with the previous two cell blocks to calculate the influence count. 
# It combines the method of calculating the spread using spreading model as defined in the NDlib package, witht the different heuristic approaches.
average = []
for index in range(7):
    # model = ep.IndependentCascadesModel(G2)
    # model = ep.SIRModel(G2)
    # model = ep.SIModel(G2)
    model = ep.ThresholdModel(G2_copy)
    cfg = mc.Configuration()
    # cfg.add_model_parameter('beta', 0.01)
    # cfg.add_model_parameter('gamma', 0.005)
    # cfg.add_model_parameter('lambda', 0.005)
    infected_nodes = first_100_after
    cfg.add_model_initial_configuration("Infected", infected_nodes)
    # cfg.add_model_parameter("fraction_infected", 0.05)
    model.set_initial_status(cfg)
    threshold = 0.1
    # for e in G2.edges():
    #     cfg.add_edge_configuration("threshold", e, threshold)
    for n in G2_copy.nodes():
        cfg.add_node_configuration("threshold", n, random.uniform(0,1))

    # iterations = model.iteration_bunch(500, True, True)
    # trends = model.build_trends(iterations)
    # viz = DiffusionTrend(model, trends)
    # p = viz.plot(width=800, height=800)
    # show(p)
    list = []
    for i in range(0,1000):
        iterations = model.iteration_bunch(1)

        dict = iterations[0]
        for i in dict:
            k = (i,dict[i])
            list.append(k)
    list2 = []
    for i in range(len(list)):
        if list[i][0] == 'status':
            list2.append(list[i][1])
    list4 = []
    for t in range(len(list2)):
        for f in list2[t]:
            if list2[t][f] == 1:
                list4.append(f)
    # print("================================")
    # print(f"Results for iteration {index+1}:")
    # print("Total nodes infected: ", len(list4), "out of", len(G2.nodes()))
    percentage = (len(list4)/len(G2_copy.nodes()))*100
    # print("Percentage of total infected nodes with current seed set: ", percentage, "%")
    # print("================================")
    average.append(percentage)
print("The average percentage is: ", sum(average)/len(average), "%")

The average percentage is:  24.25001898686109 %


In [13]:
# ICM
# This code can be used with the previous two cell blocks to calculate the influence count. 
# It combines the method of calculating the spread using spreading model as defined in the NDlib package, witht the different heuristic approaches.
average = []
for index in range(7):
    model = ep.IndependentCascadesModel(G2_copy)
    # model = ep.SIRModel(G2)
    # model = ep.SIModel(G2)
    # model = ep.ThresholdModel(G2_copy_comm_fair)
    cfg = mc.Configuration()
    # cfg.add_model_parameter('beta', 0.01)
    # cfg.add_model_parameter('gamma', 0.005)
    # cfg.add_model_parameter('lambda', 0.005)
    infected_nodes = first_100_after
    cfg.add_model_initial_configuration("Infected", infected_nodes)
    # cfg.add_model_parameter("fraction_infected", 0.05)
    model.set_initial_status(cfg)
    threshold = 0.1
    for e in G2_copy.edges():
        cfg.add_edge_configuration("threshold", e, random.uniform(0,0.5))
    # for n in G2.nodes():
    #     cfg.add_node_configuration("threshold", n, threshold)

    # iterations = model.iteration_bunch(500, True, True)
    # trends = model.build_trends(iterations)
    # viz = DiffusionTrend(model, trends)
    # p = viz.plot(width=800, height=800)
    # show(p)
    list = []
    for i in range(0,1000):
        iterations = model.iteration_bunch(1)
        # print(iterations)

        dict = iterations[0]
        for i in dict:
            k = (i,dict[i])
            list.append(k)
    list2 = []
    for i in range(len(list)):
        if list[i][0] == 'status':
            list2.append(list[i][1])
    list4 = []
    for t in range(len(list2)):
        for f in list2[t]:
            if list2[t][f] == 1:
                list4.append(f)
    # print("================================")
    # print(f"Results for iteration {index+1}:")
    # print("Total nodes infected: ", len(list4), "out of", len(G2.nodes()))
    percentage = (len(list4)/len(G2_copy.nodes()))*100
    # print("Percentage of total infected nodes with current seed set: ", percentage, "%")
    # print("================================")
    average.append(percentage)
print("The average percentage is: ", sum(average)/len(average), "%")

The average percentage is:  15.967950178476496 %


In [ ]:
# ICM
# This code can be used with the previous two cell blocks to calculate the influence count. 
# It combines the method of calculating the spread using spreading model as defined in the NDlib package, witht the different heuristic approaches.
average = []
for index in range(7):
    model = ep.IndependentCascadesModel(G2_copy)
    # model = ep.SIRModel(G2)
    # model = ep.SIModel(G2)
    # model = ep.ThresholdModel(G2_copy_comm_fair)
    cfg = mc.Configuration()
    # cfg.add_model_parameter('beta', 0.01)
    # cfg.add_model_parameter('gamma', 0.005)
    # cfg.add_model_parameter('lambda', 0.005)
    infected_nodes = first_100_after
    cfg.add_model_initial_configuration("Infected", infected_nodes)
    # cfg.add_model_parameter("fraction_infected", 0.05)
    model.set_initial_status(cfg)
    threshold = 0.1
    for e in G2_copy.edges():
        cfg.add_edge_configuration("threshold", e, 0.1)
    # for n in G2.nodes():
    #     cfg.add_node_configuration("threshold", n, threshold)

    # iterations = model.iteration_bunch(500, True, True)
    # trends = model.build_trends(iterations)
    # viz = DiffusionTrend(model, trends)
    # p = viz.plot(width=800, height=800)
    # show(p)
    list = []
    for i in range(0,50):
        iterations = model.iteration_bunch(200)

        dict = iterations[0]
        for i in dict:
            k = (i,dict[i])
            list.append(k)
    list2 = []
    for i in range(len(list)):
        if list[i][0] == 'status':
            list2.append(list[i][1])
    list4 = []
    for t in range(len(list2)):
        for f in list2[t]:
            if list2[t][f] == 1:
                list4.append(f)
    # print("================================")
    # print(f"Results for iteration {index+1}:")
    # print("Total nodes infected: ", len(list4), "out of", len(G2.nodes()))
    percentage = (len(list4)/len(G2_copy.nodes()))*100
    # print("Percentage of total infected nodes with current seed set: ", percentage, "%")
    # print("================================")
    average.append(percentage)
print("The average percentage is: ", sum(average)/len(average), "%")